In [ ]:
drop all cols with strings and unvaluable info like index
Always make sure it is sorted correctly by time and station
edit time of year with cos/sin "encode"
df['day_of_year'] = df['valid_time'].dt.dayofyear
?? subrtract a row in setting forecast hours ??

In [243]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [244]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import os
import datetime as dt
from tqdm import tqdm

In [245]:
def col_drop(df):
    df = df.drop(columns=["day_of_year", "flag"])
    df = df[df.columns.drop(list(df.filter(regex="time")))]
    df = df[df.columns.drop(list(df.filter(regex="station")))]
    df = df[df.columns.drop(list(df.filter(regex="tair")))]
    df = df[df.columns.drop(list(df.filter(regex="ta9m")))]
    df = df[df.columns.drop(list(df.filter(regex="td")))]
    df = df[df.columns.drop(list(df.filter(regex="relh")))]
    df = df[df.columns.drop(list(df.filter(regex="srad")))]
    df = df[df.columns.drop(list(df.filter(regex="pres")))]
    df = df[df.columns.drop(list(df.filter(regex="wspd")))]
    df = df[df.columns.drop(list(df.filter(regex="wmax")))]
    df = df[df.columns.drop(list(df.filter(regex="wdir")))]
    df = df[df.columns.drop(list(df.filter(regex="precip_total")))]
    df = df[df.columns.drop(list(df.filter(regex="snow_depth")))]

    return df

In [246]:
def get_flag(hrrr_df):
    stations_ls = hrrr_df["station"].unique()
    one_hour = dt.timedelta(hours=1)
    flag_ls = []

    for station in stations_ls:
        df = hrrr_df[hrrr_df["station"] == station]
        time_ls = df["valid_time"].tolist()
        for now, then in zip(time_ls, time_ls[1:]):
            if now + one_hour == then:
                flag_ls.append(True)
            else:
                flag_ls.append(False)

    flag_ls.append(True)
    hrrr_df["flag"] = flag_ls

    return hrrr_df

In [247]:
def nwp_error(target, station, df):
    vars_dict = {
        "t2m": "tair",
        "mslma": "pres",
    }
    nysm_var = vars_dict.get(target)

    df["target_error"] = df[f"{target}_{station}"] - df[f"{nysm_var}_{station}"]
    return df

In [248]:
def encode(data, col, max_val):
    data[col + "_sin"] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + "_cos"] = np.cos(2 * np.pi * data[col] / max_val)

    return data

In [249]:
df = pd.read_parquet(
    "/home/aevans/nwp_bias/src/machine_learning/data/clean_parquets/cleaned_rough_lstm_nysmcat_Western Plateau.parquet"
)
df = df.dropna()

In [250]:
df.head()

,valid_time,time,station_ADDI,latitude_ADDI,longitude_ADDI,t2m_ADDI,sh2_ADDI,d2m_ADDI,r2_ADDI,u10_ADDI,...,26_aspect_RAND,31_aspect_RAND,33_aspect_RAND,32_aspect_RAND,34_aspect_RAND,38_aspect_RAND,std_elev_RAND,variance_elev_RAND,skew_elev_RAND,med_dist_elev_RAND
0,2018-01-01 03:00:00,2018-01-01 01:00:00,ADDI,42.045955,-77.218867,-19.199194,0.00068,-21.993216,76.800003,3.321081,...,5.217391,0.0,0.0,0.0,0.0,0.0,77.124478,5896.461701,0.569549,405.940002
1,2018-01-01 04:00:00,2018-01-01 02:00:00,ADDI,42.045955,-77.218867,-19.118689,0.00068,-21.918570,76.500000,3.298830,...,5.217391,0.0,0.0,0.0,0.0,0.0,77.124478,5896.461701,0.569549,405.940002
2,2018-01-01 05:00:00,2018-01-01 03:00:00,ADDI,42.045955,-77.218867,-19.439062,0.00065,-22.570868,73.800003,2.992857,...,5.217391,0.0,0.0,0.0,0.0,0.0,77.124478,5896.461701,0.569549,405.940002
3,2018-01-01 06:00:00,2018-01-01 04:00:00,ADDI,42.045955,-77.218867,-19.760291,0.00063,-22.821158,74.400002,3.115693,...,5.217391,0.0,0.0,0.0,0.0,0.0,77.124478,5896.461701,0.569549,405.940002
4,2018-01-01 07:00:00,2018-01-01 05:00:00,ADDI,42.045955,-77.218867,-19.860175,0.00062,-22.993704,74.099998,3.268302,...,5.217391,0.0,0.0,0.0,0.0,0.0,77.124478,5896.461701,0.569549,405.940002


In [251]:
# columns to reintigrate back into the df after model is done running
cols_to_carry = ["valid_time", "flag"]

In [252]:
# features

In [253]:
# edit dataframe
df = df[df.columns.drop(list(df.filter(regex="day")))]
df = get_flag(df)
df["day_of_year"] = df["valid_time"].dt.dayofyear
df = encode(df, "day_of_year", 366)
df = nwp_error("t2m", "ADDI", df)
new_df = col_drop(df)

In [254]:
new_df

,latitude_ADDI,longitude_ADDI,t2m_ADDI,sh2_ADDI,d2m_ADDI,r2_ADDI,u10_ADDI,v10_ADDI,tp_ADDI,mslma_ADDI,...,33_aspect_RAND,32_aspect_RAND,34_aspect_RAND,38_aspect_RAND,variance_elev_RAND,skew_elev_RAND,med_dist_elev_RAND,day_of_year_sin,day_of_year_cos,target_error
0,42.045955,-77.218867,-19.199194,0.00068,-21.993216,76.800003,3.321081,-2.272873,0.001,103034.0,...,0.0,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-0.368794
1,42.045955,-77.218867,-19.118689,0.00068,-21.918570,76.500000,3.298830,-1.821096,0.000,103019.0,...,0.0,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-0.600048
2,42.045955,-77.218867,-19.439062,0.00065,-22.570868,73.800003,2.992857,-0.877062,0.000,103027.0,...,0.0,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-0.803223
3,42.045955,-77.218867,-19.760291,0.00063,-22.821158,74.400002,3.115693,-0.789671,0.000,103071.0,...,0.0,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-1.059481
4,42.045955,-77.218867,-19.860175,0.00062,-22.993704,74.099998,3.268302,-0.054269,0.001,103052.0,...,0.0,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-0.031725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22486,42.045955,-77.218867,11.485986,0.00710,8.142847,80.300003,0.073768,5.399014,0.000,100762.0,...,0.0,0.0,0.0,0.0,5896.461701,0.569549,405.940002,-0.017166,0.999853,0.660996
22487,42.045955,-77.218867,10.447198,0.00676,7.376611,80.400002,-0.416093,5.292988,0.000,100785.0,...,0.0,0.0,0.0,0.0,5896.461701,0.569549,405.940002,-0.017166,0.999853,0.637534
22488,42.045955,-77.218867,9.658258,0.00709,8.101648,88.000000,0.234075,5.295895,0.000,100879.0,...,0.0,0.0,0.0,0.0,5896.461701,0.569549,405.940002,-0.017166,0.999853,-0.462142
22489,42.045955,-77.218867,9.118555,0.00711,8.145227,92.000000,0.285457,5.515205,0.038,100849.0,...,0.0,0.0,0.0,0.0,5896.461701,0.569549,405.940002,-0.017166,0.999853,-0.493114


In [255]:
keys = new_df.keys()

for k in keys:
    print(k)

latitude_ADDI
longitude_ADDI
t2m_ADDI
sh2_ADDI
d2m_ADDI
r2_ADDI
u10_ADDI
v10_ADDI
tp_ADDI
mslma_ADDI
orog_ADDI
tcc_ADDI
asnow_ADDI
cape_ADDI
dswrf_ADDI
dlwrf_ADDI
gh_ADDI
u_total_ADDI
u_dir_ADDI
new_tp_ADDI
lat_ADDI
lon_ADDI
elev_ADDI
mslp_ADDI
11_nlcd_ADDI
21_nlcd_ADDI
22_nlcd_ADDI
23_nlcd_ADDI
24_nlcd_ADDI
31_nlcd_ADDI
41_nlcd_ADDI
42_nlcd_ADDI
43_nlcd_ADDI
52_nlcd_ADDI
71_nlcd_ADDI
81_nlcd_ADDI
82_nlcd_ADDI
90_nlcd_ADDI
95_nlcd_ADDI
19_aspect_ADDI
21_aspect_ADDI
24_aspect_ADDI
27_aspect_ADDI
28_aspect_ADDI
22_aspect_ADDI
23_aspect_ADDI
25_aspect_ADDI
26_aspect_ADDI
31_aspect_ADDI
33_aspect_ADDI
32_aspect_ADDI
34_aspect_ADDI
38_aspect_ADDI
variance_elev_ADDI
skew_elev_ADDI
med_dist_elev_ADDI
latitude
longitude
t2m
sh2
d2m
r2
u10
v10
tp
mslma
orog
tcc
asnow
cape
dswrf
dlwrf
gh
u_total
u_dir
new_tp
lat
lon
elev
mslp
11_nlcd
21_nlcd
22_nlcd
23_nlcd
24_nlcd
31_nlcd
41_nlcd
42_nlcd
43_nlcd
52_nlcd
71_nlcd
81_nlcd
82_nlcd
90_nlcd
95_nlcd
19_aspect
21_aspect
24_aspect
27_aspect
28_aspect
22

## create train and test set

In [256]:
target_sensor = "target_error"
features = list(new_df.columns.difference([target_sensor]))

forecast_lead = 12
target = f"{target_sensor}_lead_{forecast_lead}"

new_df[target] = new_df[target_sensor].shift(-forecast_lead)
new_df = new_df.iloc[:-forecast_lead]

In [257]:
new_df

,latitude_ADDI,longitude_ADDI,t2m_ADDI,sh2_ADDI,d2m_ADDI,r2_ADDI,u10_ADDI,v10_ADDI,tp_ADDI,mslma_ADDI,...,32_aspect_RAND,34_aspect_RAND,38_aspect_RAND,variance_elev_RAND,skew_elev_RAND,med_dist_elev_RAND,day_of_year_sin,day_of_year_cos,target_error,target_error_lead_12
0,42.045955,-77.218867,-19.199194,0.00068,-21.993216,76.800003,3.321081,-2.272873,0.001,103034.0,...,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-0.368794,-0.013050
1,42.045955,-77.218867,-19.118689,0.00068,-21.918570,76.500000,3.298830,-1.821096,0.000,103019.0,...,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-0.600048,0.538113
2,42.045955,-77.218867,-19.439062,0.00065,-22.570868,73.800003,2.992857,-0.877062,0.000,103027.0,...,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-0.803223,0.320462
3,42.045955,-77.218867,-19.760291,0.00063,-22.821158,74.400002,3.115693,-0.789671,0.000,103071.0,...,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-1.059481,0.073639
4,42.045955,-77.218867,-19.860175,0.00062,-22.993704,74.099998,3.268302,-0.054269,0.001,103052.0,...,0.0,0.0,0.0,5896.461701,0.569549,405.940002,0.017166,0.999853,-0.031725,-0.233670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22474,42.045955,-77.218867,7.214929,0.00491,2.928369,68.599998,0.539387,7.163856,0.000,101497.0,...,0.0,0.0,0.0,5896.461701,0.569549,405.940002,-0.017166,0.999853,-0.196832,0.660996
22475,42.045955,-77.218867,6.744348,0.00513,3.522516,74.800003,0.926868,7.843880,0.000,101424.0,...,0.0,0.0,0.0,5896.461701,0.569549,405.940002,-0.017166,0.999853,-1.117833,0.637534
22476,42.045955,-77.218867,7.081567,0.00524,3.839014,74.300003,1.633466,6.721133,0.000,101523.0,...,0.0,0.0,0.0,5896.461701,0.569549,405.940002,-0.017166,0.999853,-0.502218,-0.462142
22477,42.045955,-77.218867,6.931268,0.00525,3.896814,74.000000,1.061213,5.333986,0.000,101464.0,...,0.0,0.0,0.0,5896.461701,0.569549,405.940002,-0.017166,0.999853,0.533270,-0.493114


In [258]:
length = len(new_df)

test_len = int(length * 0.75)

df_train = new_df.iloc[:test_len].copy()
df_test = new_df.iloc[test_len:].copy()

print("Test Set Fraction", len(df_test) / len(new_df))

Test Set Fraction 0.25001112149116955


## Normalize

In [259]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev

In [260]:
for c in cols_to_carry:
    df_train[c] = df[c]
    df_test[c] = df[c]

/tmp/tmp.xNs8XNoGNT/ipykernel_2009987/3098142065.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/tmp.xNs8XNoGNT/ipykernel_2009987/3098142065.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/tmp.xNs8XNoGNT/ipykernel_2009987/3098142065.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In [261]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

## Create LSTM

In [262]:
class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.dataframe = dataframe
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[target].values).float()
        self.X = torch.tensor(dataframe[features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i):
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start : (i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0 : (i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [263]:
class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 3

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers,
        )
        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(
            self.num_layers, batch_size, self.hidden_units
        ).requires_grad_()
        c0 = torch.zeros(
            self.num_layers, batch_size, self.hidden_units
        ).requires_grad_()

        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(
            hn[0]
        ).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

In [264]:
class EarlyStopper:
    def __init__(self, patience, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [265]:
# def remove_elements_from_batch(X, y, s):
#     cond = (np.where(s))
#     return X[cond], y[cond]

In [266]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()

    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # loss
    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")
    return avg_loss


def test_model(data_loader, model, loss_function):
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    # loss
    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")

    return avg_loss

In [267]:
torch.manual_seed(101)
batch_size = 14
sequence_length = 18
learning_rate = 5e-4
num_hidden_units = 80

experiment = Experiment(
    api_key="leAiWyR5Ck7tkdiHIT7n6QWNa",
    project_name="fh_2_hrrr",
    workspace="shmaronshmevans",
)

batch_size = batch_size
sequence_length = sequence_length

train_dataset = SequenceDataset(
    df_train, target=target, features=features, sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test, target=target, features=features, sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

learning_rate = learning_rate
num_hidden_units = num_hidden_units

model = ShallowRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
early_stopper = EarlyStopper(patience=7, min_delta=0)


print("Untrained test\n--------")
test_model(test_loader, model, loss_function)
print()

for ix_epoch in range(50):
    print(f"Epoch {ix_epoch}\n---------")
    train_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)
    val_loss = test_model(test_loader, model, loss_function)
    print()
    if early_stopper.early_stop(val_loss):
        break

# Report multiple hyperparameters using a dictionary:
hyper_params = {
    "learning_rate": learning_rate,
    "steps": sequence_length,
    "batch_size": batch_size,
}
experiment.log_parameters(hyper_params)

# Seamlessly log your Pytorch model
log_model(experiment, model, model_name="exp1")
experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/shmaronshmevans/fh-2-hrrr/fa62ca78687d483885390d9bfc11a087



Features shape: torch.Size([14, 18, 562])
Target shape: torch.Size([14])
Untrained test
--------
Test loss: 1.025800741245201

Epoch 0
---------
Train loss: 0.8363225926007473
Test loss: 1.001200636459598

Epoch 1
---------
Train loss: 0.7033963551536141
Test loss: 1.0986514310931685

Epoch 2
---------
Train loss: 0.5846563657908024
Test loss: 1.1316140023574455

Epoch 3
---------
Train loss: 0.5022328639858986
Test loss: 1.1281149267168038

Epoch 4
---------
Train loss: 0.4417840479784239
Test loss: 1.1926933948080338

Epoch 5
---------
Train loss: 0.39794421123023843
Test loss: 1.1799181292565604

Epoch 6
---------
Train loss: 0.3632018240748352
Test loss: 1.269446609597375

Epoch 7
---------
Train loss: 0.33230694055371757
Test loss: 1.2883461707489408



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/shmaronshmevans/fh-2-hrrr/fa62ca78687d483885390d9bfc11a087
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [964] : (0.053031377494335175, 2.598637342453003)
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 14
COMET INFO:     learning_rate : 0.0005
COMET INFO:     steps         : 18
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment details          : 1
COMET INFO:     filename                     : 1
COMET INFO:     git metadata                 : 1
COMET INFO: 

## Evaluate Model

In [268]:
def predict(data_loader, model):
    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)

    return output


train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = "Model forecast"
df_train[ystar_col] = predict(train_eval_loader, model).numpy()
df_test[ystar_col] = predict(test_loader, model).numpy()

df_out = pd.concat((df_train, df_test))[[target, ystar_col]]

for c in df_out.columns:
    df_out[c] = df_out[c] * target_stdev + target_mean

print(df_out)

       target_error_lead_12  Model forecast
0                 -0.013050       -0.712577
1                  0.538113       -0.722718
2                  0.320462       -0.708932
3                  0.073639       -0.814323
4                 -0.233670       -0.605532
...                     ...             ...
22474              0.660996        0.908875
22475              0.637534        0.889611
22476             -0.462142        0.779963
22477             -0.493114        0.668557
22478              0.067485        0.608053

[22479 rows x 2 columns]


In [269]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"
plot_template = dict(
    layout=go.Layout(
        {"font_size": 18, "xaxis_title_font_size": 24, "yaxis_title_font_size": 24}
    )
)

fig = px.line(df_out, labels=dict(created_at="Date", value="Forecast Error"))
fig.add_vline(x=(length * 0.75), line_width=4, line_dash="dash")
fig.add_annotation(
    xref="paper", x=0.75, yref="paper", y=0.8, text="Test set start", showarrow=False
)
fig.update_layout(
    template=plot_template, legend=dict(orientation="h", y=1.02, title_text="")
)
fig.show()